# Poisson Models

Roughly speaking, Poisson models are natural choices for count data (c.f. https://data.princeton.edu/wws509/notes/#), and the main features are:
    - The data is positive
    - The variance of the data increases with the mean (often empirically true for count data -- larger counts fluctuate more wildly)
    - The events producing the counts are independent. Example: nuclear decay events that do not affect each other. Counterexample: nuclear chain reactions, where one decay causes many more.
    
More specifically, the likelihood of observing a value $y$ is assumed to follow a Poisson distribution:

\begin{equation}
p(y|\mu) = \frac{e^{-\mu} \mu^{y}}{y!}
\end{equation}

Where the only parameter, $\mu$, is both the mean and the variance of the Poisson distribution. The model acquires additional structure when $\mu$ is assumed to be a function of some explanatory variables, $x$, i.e. $\mu = f(x)$. When $f(x)$ is a linear function, then the Poisson Model becomes a generalized linear model (GLM). The familiar way of writing that is $\mu = \beta x$.

## Analogy to Ordinary Least Squares Regression
To anchor intuition in familiar territory, consider ordinary least squares regression. The familiar form for the model is:

\begin{equation}
\begin{array}{rl}
y &= a + \mathbf{b x} + \epsilon \\
&= \mathbf{\beta x} + \epsilon
\end{array}
\end{equation}

Where the constant $a$ was absorbed into the coefficient vector $\mathbf{\beta}$ in the second line, and $\mathbf{x} \rightarrow [1,\mathbf{x}]$. $\epsilon$ is an error term that is assumed to have normal distribution with zero mean, i.e. $\epsilon \sim \mathscr{N}(0,\sigma)$. It's a bit unnatural, but this can be rewritten, absorbing the parameters into the random term:

\begin{equation}
y = 0 + \epsilon'
\end{equation}

With $\epsilon' \sim \mathscr{N}(\mu = \mathbf{\beta x},\sigma)$. Now what if the fluctuations aren't normally distributed about the mean, but they are Poisson distributed about the mean? In that case, $\epsilon' \sim \mathrm{Poisson}(\mu = \mathbf{\beta x})$. 

Because of this, Poisson regression might be referred to as "regression with Poisson error".

## Generalization: Multivariate Poisson Model
For least squared regression, it's totally common to look at multivariate models with interesting codependence structure captured by a covariance matrix. In analogy to that, there is multivariate Poisson Regression. The math looks quite different. 

http://www2.stat-athens.aueb.gr/~karlis/multivariate%20Poisson%20models.pdf

## Generalization:  Nonlinear $\mu = f(\mathbf{x})$
The Poisson-ness only has to do with how the data fluctuates about the mean. How the mean is expected to depend on the explanatory variables is another question.